In [2]:
#!/usr/bin/env python3

import csv
import json
import nilearn
import numpy as np
import struct
from chardet import UniversalDetector as udet
from io import StringIO
from os.path import expanduser as xpu
from os import listdir as ls
from os.path import join as pjoin
from typing import Union
from zipfile import ZipFile
from tqdm import tqdm

from cimaq_utils import get_cimaq_dir_paths
from sniffbytes import flatten
from sniffbytes import loadfiles
from sniffbytes import loadimages
from cimaq_utils import repair_enc_task
from cimaq_utils import fetch_cimaq

from json_read import json_read

from sniffbytes import *


from removeEmptyFolders import removeEmptyFolders
from multiple_replace import multiple_replace
cimaq_dir = '~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives'
otherdir = '/home/fnadeau/projects/rrg-pbellec/fnadeau/cimaq_03-19/cimaq_03-19/sub-127228/ses-4/anat'
dlst = get_cimaq_dir_paths(cimaq_dir)[0]#[['confdir', 'events_dir']]


In [3]:
dlst.T

,suffixes,fpaths
cimaq_dir,~/../../media/francois/seagate_1tb/cimaq_03-19...,/home/francois/../../media/francois/seagate_1t...
stereonl_dir,anat,/home/francois/../../media/francois/seagate_1t...
behavioral_dir,task_retrieval_bahvioral,/home/francois/../../media/francois/seagate_1t...
confonunds_dir,confounds,/home/francois/../../media/francois/seagate_1t...
events_dir,task_encoding_events,/home/francois/../../media/francois/seagate_1t...
func_dir,fmri/resample,/home/francois/../../media/francois/seagate_1t...
temp_events_dir,cleaned_events,/home/francois/../../media/francois/seagate_1t...
mean_motion,participants/MotionResults/fMRI_meanMotion.tsv,/home/francois/../../media/francois/seagate_1t...
mean_file,participants/MemoTaskParticipantFile.tsv,/home/francois/../../media/francois/seagate_1t...
mean_npsych,participants/Neuropsych/ALL_Neuropsych_scores.tsv,/home/francois/../../media/francois/seagate_1t...


In [43]:
# def load_tsvs(cimaq_dir):
#     qc_ok = sorted([str(itm[0]) for itm in
#                 pd.read_csv(get_cimaq_dir_paths(
#                     cimaq_dir)[0].mean_qc.fpaths, sep='\t').values])
#     qc_ok
#     to_exclude = df(sorted([(str(bname(itm).split('_')[0]),
#                          str(bname(itm).split('_')[1]), itm) for itm in
#                         loadimages(get_cimaq_dir_paths(
#                             cimaq_dir)[0].zeprimes.fpaths)
#                         if str(bname(itm).split('_')[1]) not in qc_ok]),
#                    columns = ['pscid', 'dccid', 'fpaths'])`.set_index(
#                        'dccid').sort_index().reset_index().fpaths.tolist()

#     repair_dataset(get_cimaq_dir_paths(cimaq_dir)[0].zeprimes.fpaths,
#                    get_cimaq_dir_paths(cimaq_dir)[0].temp_events_dir.fpaths,
#                    exclude = ['pratique', 'practice', '.pdf', '.edat2'] + to_exclude)
#     allids = repair_enc_task(cimaq_dir)
#     pscids, dccids, subids = allids.pscid, allids.dccid, allids.subids
#     cimaq = pd.concat([subids, pscids, dccids] + \
#                        [loadfiles(loadimages(row[1].fpaths)).dropna(axis = 0).T
#                         for row in get_cimaq_dir_paths(cimaq_dir)[0].T.iloc[2: 4].iterrows()]).T
#     test = dict((grp, dict((sgrp, cimaq.groupby('subids').get_group(grp).groupby(
#                'fname').get_group(sgrp)) for sgrp in allscans.groupby(
#                    'subids').get_group(grp).groupby('fname').groups))
#                 for grp in allscans.groupby('subids').groups)
# #     cimaq = pd.concat([subids, pscids, dccids] + \
# #                       [loadfiles(loadimages(cimaqrow[1].fpaths)).dropna(axis = 0).rename(
# #                 columns = {'fpaths': cimaqrow[1].suffixes})#['fpaths']
# #                                   for cimaqrow in get_cimaq_dir_paths(cimaq_dir)[0].T.iloc[1: 6].iterrows()],
# #                       axis = 1).dropna(axis = 0).T.reset_index(drop = True).T
# #     cimaq = cimaq.rename(columns = {0: 'subid', 1: 'pscid', 2: 'dccid', 3: 'stereonl', 4: 'behavioral',
# #                                     5: 'confounds', 6: 'events', 7:'func'})
#     return test
# cimaq = load_tsvs(cimaq_dir)

sniffing: 100%|██████████| 94/94 [00:03<00:00, 31.23it/s]
repairing: 284it [00:01, 257.21it/s]
saving: 284it [00:00, 631.81it/s]
encoding_task: 94it [00:01, 75.75it/s]


KeyError: 'subids'

In [109]:
def fetch_cimaq(cimaq_dir):
    qc_ok = sorted([str(itm[0]) for itm in
                pd.read_csv(get_cimaq_dir_paths(
                    cimaq_dir)[0].mean_qc.fpaths, sep='\t').values])
    qc_ok
    to_exclude = df(sorted([(str(bname(itm).split('_')[0]),
                         str(bname(itm).split('_')[1]), itm) for itm in
                        loadimages(get_cimaq_dir_paths(
                            cimaq_dir)[0].zeprimes.fpaths)
                        if str(bname(itm).split('_')[1]) not in qc_ok]),
                   columns = ['pscid', 'dccid', 'fpaths']).set_index(
                       'dccid').sort_index().reset_index().fpaths.tolist()

    repair_dataset(get_cimaq_dir_paths(cimaq_dir)[0].zeprimes.fpaths,
                   get_cimaq_dir_paths(cimaq_dir)[0].temp_events_dir.fpaths,
                   exclude = ['pratique', 'practice', '.pdf', '.edat2'] + to_exclude)
    allids = repair_enc_task(cimaq_dir)
    pscids, dccids, subids = allids.pscid, allids.dccid, allids.subids
    cimaq = pd.concat([subids, pscids, dccids] + [loadfiles(loadimages(cimaqpath)).dropna(axis = 0)['fpaths']
                                  for cimaqpath in get_cimaq_dir_paths(cimaq_dir)[0].loc['fpaths'][2: 5]],
                      axis = 1).dropna(axis = 0).T.reset_index(drop = True).T
    cimaq = cimaq.rename(columns = {0: 'subid', 1: 'pscid', 2: 'dccid', 3: 'behavioral', 4: 'confounds',
                                    5: 'events'})
    return cimaq.set_index('dccid').sort_index()#.reset_index(drop = False)

In [110]:
cimaq = fetch_cimaq(cimaq_dir)
#cimaq.sort_values('dccid').set_index('dccid', drop = True).reset_index()

sniffing: 100%|██████████| 94/94 [00:03<00:00, 30.11it/s]
repairing: 284it [00:01, 246.21it/s]
saving: 284it [00:00, 2726.72it/s]
encoding_task: 94it [00:01, 70.42it/s]


In [103]:
cimaq

,subid,pscid,behavioral,confounds,events
dccid,,,,,
108391,sub-4509950-108391,4509950,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
120839,sub-7424803-120839,7424803,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
122922,sub-7874568-122922,7874568,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
127228,sub-3865361-127228,3865361,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
139593,sub-5760364-139593,5760364,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
...,...,...,...,...,...
979001,sub-7516889-979001,7516889,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
983291,sub-3874544-983291,3874544,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
988602,sub-6633412-988602,6633412,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...


In [105]:
def load_scans(cimaq_dir):
    cimaq = fetch_cimaq(cimaq_dir)

#     cimaq = pd.concat([subids, pscids, dccids] + \
#                   [loadfiles(loadimages(cimaqrow[1].fpaths)).dropna(axis = 0).rename(
#             columns = {'fpaths': cimaqrow[1].suffixes})#['fpaths']
#                               for cimaqrow in get_cimaq_dir_paths(cimaq_dir)[0].T.iloc[1: 6].iterrows()],
#                   axis = 1).dropna(axis = 0).T.reset_index(drop = True).T
    
    allscans = pd.concat([loadfiles(loadimages(pjoin(dname(xpu(cimaq_dir)),
                                                     sub))).dropna(axis = 0).T
                          for sub in [itm for itm in ls(dname(xpu(cimaq_dir)))
                        if itm.startswith('sub-')]], axis = 1, dtypes = str).T
    allscans[['dccid', 'modality', 'general']] = [(bname(row[1].fpaths).split('_')[0].split('-')[1],
                                                   bname(row[1].fpaths).split('_')[-1],
                                                   bname(dname(row[1].fpaths)))
                         for row in allscans.iterrows()]

    test = dict((grp, dict((sgrp, allscans.groupby('dccid').get_group(grp).groupby(
               'general').get_group(sgrp)) for sgrp in allscans.groupby(
                   'dccid').get_group(grp).groupby('general').groups))
                for grp in allscans.groupby('dccid').groups)
    allscans = df.from_dict(test, orient = 'index').sort_index()
    indexes = set.intersection(set(cimaq.index), set(allscans.index)) 
    return pd.concat([allscans.loc[indexes], cimaq.loc[indexes]], axis = 0)
test = load_scans(cimaq_dir)

sniffing: 100%|██████████| 94/94 [00:03<00:00, 29.09it/s]
repairing: 284it [00:01, 247.60it/s]
saving: 284it [00:00, 3031.52it/s]
encoding_task: 94it [00:01, 74.45it/s]


In [106]:
display(test)

,anat,dwi,fmap,func,subid,pscid,behavioral,confounds,events
936730,fname ext \ 0 sub...,fname ext \ 10 sub-9...,fname ext \ 14...,fname ext...,NaN,NaN,NaN,NaN,NaN
668786,fname ext \ 0 ...,fname ext \ 6 sub-668...,fname ext \ 10...,fname ext...,NaN,NaN,NaN,NaN,NaN
462345,fname ext \ 0 sub...,fname ext \ 10 sub-4...,fname ext \ 14 ...,fname ext...,NaN,NaN,NaN,NaN,NaN
271596,fname ext \ 0 sub...,fname ext \ 10 sub-2...,fname ext \ 14...,fname ext...,NaN,NaN,NaN,NaN,NaN
520377,fname ext \ 0 sub...,fname ext \ 10 sub-5...,fname ext \ 14...,fname ext...,NaN,NaN,NaN,NaN,NaN
229301,fname ext \ 0 sub...,fname ext \ 10 sub-2...,fname ext \ 14...,fname ext...,NaN,NaN,NaN,NaN,NaN
258912,fname ext \ 0 sub...,fname ext \ 10 sub-2...,fname ext \ 14...,fname ext...,NaN,NaN,NaN,NaN,NaN
763590,fname ext \ 0 sub...,fname ext \ 10 sub-7...,fname ext \ 14...,fname ext...,NaN,NaN,NaN,NaN,NaN
370092,fname ext \ 0 sub...,fname ext \ 10 sub-3...,fname ext \ 14...,fname ext...,NaN,NaN,NaN,NaN,NaN
255499,fname ext \ 0 sub...,fname ext \ 10 sub-2...,fname ext \ 14...,fname ext...,NaN,NaN,NaN,NaN,NaN


In [ ]:
list(test2)

In [ ]:
test2.iloc[0].fmap

In [ ]:
# params = [pd.concat([row[1][['fname']], df.from_dict(json_read(row[1].fpaths, 'r'), orient = 'index')])
#           for row in test[0].iterrows()
#           if row[1].ext == '.json']
# # allparams = pd.concat(((itm.groupby('fname').get_group(grp)
# #                  for grp in itm.groupby('fname').groups)
# #              for itm in allparams), axis = 1)
# # pd.concat(params, axis =1).T.sort_values('fname')
# pd.concat(params, axis = 1).T.set_index('fname')

In [ ]:
def _update_timestamp(path: os.PathLike, set_new: bool) -> None:
    """
    Context manager to set the timestamp of the path to plus or
    minus a fixed delta, regardless of modifications within the context.

    if set_new is True, the delta is added. Otherwise, the delta is subtracted.
    """
    stats = os.stat(path)
    if set_new:
        new_timestamp = (stats.st_atime_ns + _TIMESTAMP_DELTA, stats.st_mtime_ns + _TIMESTAMP_DELTA)
    else:
        new_timestamp = (stats.st_atime_ns - _TIMESTAMP_DELTA, stats.st_mtime_ns - _TIMESTAMP_DELTA)
    try:
        yield
    finally:
        os.utime(path, ns=new_timestamp)


# Public Methods 